In [1]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("article_new.csv")
print(len(df))
df = df.iloc[:, 3:5]
df.head()

83


,Insight,Insight Type
0,The Utah Jazz have helped Lauri Markkanen unlo...,"Single Player Performance, Strategy and Tactic"
1,Division and conference makes it easy for Mave...,Strategy and Tactic
2,Kevin Durant had to leave the game after Jimmy...,Accident
3,"The Lakers hustled more, but the Mavericks sho...",Whole Game Review
4,"Without Kevin Durant, Kyrie Irving is aggressi...","Single Player Performance, Strategy and Tactic"


In [4]:
types = df['Insight Type'].values
types = [insight_type.split(", ") for insight_type in types]
type_dictionary = {}
for type_list in types:
    for type_ in type_list:
        # print(type_)
        if type_ not in type_dictionary:
            type_dictionary[type_] = 1
        else:
            type_dictionary[type_] += 1

type_dictionary

{'Single Player Performance': 51,
 'Strategy and Tactic': 26,
 'Accident': 6,
 'Whole Game Review': 22,
 'About Referee': 4}

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
multi_label = MultiLabelBinarizer(classes=sorted(type_dictionary.keys()))
y_train = multi_label.fit_transform(types)
print(y_train[0])

[0 0 1 1 0]


In [6]:
import time

In [7]:
prompt = """
Decide whether an uterance describing the data fact type is value, difference, proportion, trend, categorization, distribution, rank, association, extreme, or outlier.
Output labels only.
 """

def getLabel(input):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt = prompt + input,
        temperature=0,
        max_tokens=128,
        frequency_penalty=0,
        presence_penalty=0
    )
    message = response["choices"][0]["text"].strip()
    # print(message)
    time.sleep(2)
    
    return message

虽然并行效率高，但是openAI会限速，不能使用并行代码来处理。

In [8]:
insights = df['Insight'].values

# y_predict = [getLabel(insight).split(", ") for insight in insights]
# y_predict = multi_label.fit_transform(y_predict)

In [9]:
y_predict = []
for insight in insights:
    print(insight)
    while (True):
        try:
            label = getLabel(insight)
            break
        except Exception as exc: 
            print(exc)
            time.sleep(1)
    labels = label.split(", ")
    print(labels)
    y_predict.append(labels)

y_predict = multi_label.fit_transform(y_predict)
print(y_predict[1])

The Utah Jazz have helped Lauri Markkanen unlock his offensive potential, becoming the kind of star they build a new foundation around.
['No labels']
Division and conference makes it easy for Maverick to win
['Categorization']
Kevin Durant had to leave the game after Jimmy Butler landed on his knee on this play.
['Outlier']
The Lakers hustled more, but the Mavericks shooting prevailed and this helps Mavericks won the game.
['Difference']
Without Kevin Durant, Kyrie Irving is aggressive and shot in the form of an ill-advised 30-footer
['Difference']
Nikola Jokic made a game-winning three-point jumper with .2 seconds on the clock to secure the victory for Denver Nuggets.
['Extreme']
Brooklyn Nets’ challenge of holding the fort without Kevin Durant became even more challenging.
['Outlier']
After Houston Rockets tore through their defense early, Los Angeles Clippers settled down and and outscored Houston Rockets in the fourth.
['Difference']
Wood is playing at All-Star level.
['Value']
Gab

d:\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:875: UserWarning: unknown class(es) ['Association', 'Categorization', 'Difference', 'Extreme', 'No applicable data fact type', 'No applicable fact type', 'No label', 'No labels', 'Outlier', 'Proportion', 'Trend', 'Value', 'than usual.\n\nDifference', 'trend'] will be ignored
  warnings.warn(
